In [1]:
"""Semantic search pipeline using DualNode for batch-optimized encoding.

This script demonstrates:
- Encoding k passages (corpus)
- Encoding m queries
- Finding nearest neighbors for each query
- Using DualNode for batch optimization
- Testing with both SequentialEngine and DaftEngine
"""

import time
from typing import List

import numpy as np

from hypernodes import DualNode, Pipeline

try:
    from daft import Series

    from hypernodes.engines import DaftEngine

    DAFT_AVAILABLE = True
except ImportError:
    DAFT_AVAILABLE = False
    Series = List
    print("⚠️  Daft not available - will only test SequentialEngine")


# ============================================================================
# Mock Encoder (simulates sentence transformers)
# ============================================================================


class MockEncoder:
    """Simulates a sentence encoder with batch capability."""

    def __init__(self, model_name: str, embedding_dim: int = 384):
        self.model_name = model_name
        self.embedding_dim = embedding_dim
        print(f"[INIT] Loading encoder: {model_name} (dim={embedding_dim})")

    def encode(self, text: str) -> List[float]:
        """Encode single text (deterministic based on hash)."""
        # Deterministic encoding based on text hash
        np.random.seed(hash(text) % (2**32))
        return np.random.randn(self.embedding_dim).tolist()

    def encode_batch(self, texts: Series) -> Series:
        """Encode batch of texts efficiently."""
        text_list = texts.to_pylist() if hasattr(texts, "to_pylist") else texts
        embeddings = [self.encode(text) for text in text_list]

        if hasattr(Series, "from_pylist"):
            return Series.from_pylist(embeddings)
        return embeddings


# ============================================================================
# DualNode Functions: Encoding
# ============================================================================


def encode_text(text: str, encoder: MockEncoder) -> List[float]:
    """Encode single text."""
    return encoder.encode(text)


def encode_text_batch(texts: Series, encoder: MockEncoder) -> Series:
    """Encode batch of texts (optimized)."""
    return encoder.encode_batch(texts)


# Create DualNode for encoding
encode_node = DualNode(
    output_name="embedding",
    singular=encode_text,
    batch=encode_text_batch,
)


# ============================================================================
# Regular function: Find nearest neighbors
# ============================================================================


def cosine_similarity(vec1: List[float], vec2: List[float]) -> float:
    """Compute cosine similarity between two vectors."""
    vec1_arr = np.array(vec1)
    vec2_arr = np.array(vec2)

    dot_product = np.dot(vec1_arr, vec2_arr)
    norm1 = np.linalg.norm(vec1_arr)
    norm2 = np.linalg.norm(vec2_arr)

    if norm1 == 0 or norm2 == 0:
        return 0.0

    return float(dot_product / (norm1 * norm2))


def find_nearest_passages(
    embedding: List[float],
    passage_embeddings: List[List[float]],
    passages: List[str],
    top_k: int = 3,
) -> List[dict]:
    """Find top-k nearest passages for a single query."""
    # Compute similarities
    similarities = [
        cosine_similarity(embedding, passage_emb) for passage_emb in passage_embeddings
    ]

    # Get top-k indices
    top_indices = np.argsort(similarities)[::-1][:top_k]

    # Return results
    results = []
    for idx in top_indices:
        results.append(
            {
                "passage": passages[idx],
                "score": float(similarities[idx]),
                "rank": len(results) + 1,
            }
        )

    return results


def find_nearest_passages_batch(
    embeddings: Series,
    passage_embeddings: List[List[float]],
    passages: List[str],
    top_k: int = 3,
) -> Series:
    """Find top-k nearest passages for batch of queries."""
    emb_list = (
        embeddings.to_pylist() if hasattr(embeddings, "to_pylist") else embeddings
    )

    results = [
        find_nearest_passages(emb, passage_embeddings, passages, top_k)
        for emb in emb_list
    ]

    if hasattr(Series, "from_pylist"):
        return Series.from_pylist(results)
    return results


# Create DualNode for nearest neighbor search
search_node = DualNode(
    output_name="search_results",
    singular=find_nearest_passages,
    batch=find_nearest_passages_batch,
)


# ============================================================================
# Test Data
# ============================================================================


def create_test_data(num_passages: int = 10, num_queries: int = 3):
    """Create test passages and queries."""
    passages = [
        "Machine learning is a subset of artificial intelligence.",
        "Deep learning uses neural networks with multiple layers.",
        "Natural language processing helps computers understand text.",
        "Computer vision enables machines to interpret visual data.",
        "Reinforcement learning trains agents through rewards.",
        "Supervised learning uses labeled training data.",
        "Unsupervised learning finds patterns in unlabeled data.",
        "Transfer learning reuses pre-trained models.",
        "Generative AI creates new content from learned patterns.",
        "Large language models process and generate human-like text.",
    ][:num_passages]

    queries = [
        "What is deep learning?",
        "How do computers understand language?",
        "Tell me about AI training methods.",
    ][:num_queries]

    return passages, queries


# ============================================================================
# Pipeline Creation
# ============================================================================


def create_encoding_pipeline():
    """Create pipeline for encoding text."""
    return Pipeline(nodes=[encode_node])


def create_search_pipeline():
    """Create pipeline for encoding query and searching."""
    return Pipeline(nodes=[encode_node, search_node])


# ============================================================================
# Test Functions
# ============================================================================


def test_sequential_engine():
    """Test with SequentialEngine."""
    print("\n" + "=" * 70)
    print("Testing with SequentialEngine")
    print("=" * 70)

    passages, queries = create_test_data(num_passages=10, num_queries=3)
    encoder = MockEncoder(model_name="mock-encoder-v1")

    # Step 1: Encode all passages
    print(f"\n[STEP 1] Encoding {len(passages)} passages...")
    encoding_pipeline = create_encoding_pipeline()

    start_time = time.time()
    passage_results = encoding_pipeline.map(
        inputs={"text": passages, "encoder": encoder}, map_over="text"
    )
    encoding_time = time.time() - start_time

    passage_embeddings = [r["embedding"] for r in passage_results]
    print(f"✅ Encoded {len(passage_embeddings)} passages in {encoding_time:.3f}s")

    # Step 2: Search for each query
    print(f"\n[STEP 2] Processing {len(queries)} queries...")
    search_pipeline = create_search_pipeline()
    display(search_pipeline.visualize())

    start_time = time.time()
    search_results = search_pipeline.map(
        inputs={
            "text": queries,
            "encoder": encoder,
            "passage_embeddings": passage_embeddings,
            "passages": passages,
            "top_k": 3,
        },
        map_over="text",
    )
    search_time = time.time() - start_time

    print(f"✅ Searched {len(queries)} queries in {search_time:.3f}s")

    # Display results
    print("\n" + "-" * 70)
    print("Search Results:")
    print("-" * 70)
    for i, (query, result) in enumerate(zip(queries, search_results)):
        print(f"\nQuery {i + 1}: {query}")
        print(
            f"Embedding: [{result['embedding'][0]:.3f}, {result['embedding'][1]:.3f}, ...]"
        )
        print("Top matches:")
        for match in result["search_results"]:
            print(
                f"  {match['rank']}. [{match['score']:.3f}] {match['passage'][:60]}..."
            )

    print("\n" + "=" * 70)
    print(f"Total time: {encoding_time + search_time:.3f}s")
    print("=" * 70)

    return passage_embeddings, search_results


def test_daft_engine():
    """Test with DaftEngine."""
    if not DAFT_AVAILABLE:
        print("\n⚠️  Skipping DaftEngine test (daft not installed)")
        return None, None

    print("\n" + "=" * 70)
    print("Testing with DaftEngine")
    print("=" * 70)

    passages, queries = create_test_data(num_passages=10, num_queries=3)
    encoder = MockEncoder(model_name="mock-encoder-v1")

    daft_engine = DaftEngine()

    # Step 1: Encode all passages
    print(f"\n[STEP 1] Encoding {len(passages)} passages with DaftEngine...")
    encoding_pipeline = create_encoding_pipeline()
    encoding_pipeline.engine = daft_engine

    start_time = time.time()
    passage_results = encoding_pipeline.map(
        inputs={"text": passages, "encoder": encoder}, map_over="text"
    )
    encoding_time = time.time() - start_time

    passage_embeddings = [r["embedding"] for r in passage_results]
    print(f"✅ Encoded {len(passage_embeddings)} passages in {encoding_time:.3f}s")
    print("   (Used batch function - check for fewer encoder calls!)")

    # Step 2: Search for each query
    print(f"\n[STEP 2] Processing {len(queries)} queries with DaftEngine...")
    search_pipeline = create_search_pipeline()
    search_pipeline.engine = daft_engine

    start_time = time.time()
    search_results = search_pipeline.map(
        inputs={
            "text": queries,
            "encoder": encoder,
            "passage_embeddings": passage_embeddings,
            "passages": passages,
            "top_k": 3,
        },
        map_over="text",
    )
    search_time = time.time() - start_time

    print(f"✅ Searched {len(queries)} queries in {search_time:.3f}s")

    # Display results
    print("\n" + "-" * 70)
    print("Search Results:")
    print("-" * 70)
    for i, (query, result) in enumerate(zip(queries, search_results)):
        print(f"\nQuery {i + 1}: {query}")
        print(
            f"Embedding: [{result['embedding'][0]:.3f}, {result['embedding'][1]:.3f}, ...]"
        )
        print("Top matches:")
        for match in result["search_results"]:
            print(
                f"  {match['rank']}. [{match['score']:.3f}] {match['passage'][:60]}..."
            )

    print("\n" + "=" * 70)
    print(f"Total time: {encoding_time + search_time:.3f}s")
    print("=" * 70)

    return passage_embeddings, search_results


def verify_consistency(seq_results, daft_results):
    """Verify both engines produce same results."""
    if seq_results is None or daft_results is None:
        return

    print("\n" + "=" * 70)
    print("Verifying Consistency Between Engines")
    print("=" * 70)

    # Compare passage embeddings
    seq_passages, seq_searches = seq_results
    daft_passages, daft_searches = daft_results

    print(f"\n✓ Both encoded {len(seq_passages)} passages")

    # Check if embeddings match
    all_match = True
    for i, (seq_emb, daft_emb) in enumerate(zip(seq_passages, daft_passages)):
        if not np.allclose(seq_emb, daft_emb, atol=1e-6):
            print(f"  ⚠️  Passage {i} embeddings differ!")
            all_match = False

    if all_match:
        print("  ✅ All passage embeddings match!")

    # Check search results
    print(f"\n✓ Both processed {len(seq_searches)} queries")

    for i, (seq_res, daft_res) in enumerate(zip(seq_searches, daft_searches)):
        # Check query embeddings
        if not np.allclose(seq_res["embedding"], daft_res["embedding"], atol=1e-6):
            print(f"  ⚠️  Query {i} embedding differs!")
            all_match = False

        # Check search results
        seq_passages_found = [r["passage"] for r in seq_res["search_results"]]
        daft_passages_found = [r["passage"] for r in daft_res["search_results"]]

        if seq_passages_found != daft_passages_found:
            print(f"  ⚠️  Query {i} search results differ!")
            all_match = False

    if all_match:
        print("  ✅ All search results match!")

    print("\n" + "=" * 70)
    if all_match:
        print("🎉 SUCCESS: Both engines produce identical results!")
    else:
        print("⚠️  WARNING: Some differences detected")
    print("=" * 70)


# ============================================================================
# Main
# ============================================================================

if __name__ == "__main__":
    print("=" * 70)
    print("Semantic Search Pipeline with DualNode")
    print("=" * 70)
    print("\nThis demonstrates:")
    print("  - Batch-optimized encoding using DualNode")
    print("  - Semantic search with nearest neighbor retrieval")
    print("  - Comparison of SequentialEngine vs DaftEngine")

    # Test with SequentialEngine
    seq_results = test_sequential_engine()

    # Test with DaftEngine
    # daft_results = test_daft_engine()

    # Verify consistency
    # verify_consistency(seq_results, daft_results)

    print("\n" + "=" * 70)
    print("✅ All tests completed!")
    print("=" * 70)


Semantic Search Pipeline with DualNode

This demonstrates:
  - Batch-optimized encoding using DualNode
  - Semantic search with nearest neighbor retrieval
  - Comparison of SequentialEngine vs DaftEngine

Testing with SequentialEngine
[INIT] Loading encoder: mock-encoder-v1 (dim=384)

[STEP 1] Encoding 10 passages...
✅ Encoded 10 passages in 0.017s

[STEP 2] Processing 3 queries...


✅ Searched 3 queries in 0.014s

----------------------------------------------------------------------
Search Results:
----------------------------------------------------------------------

Query 1: What is deep learning?
Embedding: [-0.951, 0.491, ...]
Top matches:
  1. [0.085] Natural language processing helps computers understand text....
  2. [0.037] Transfer learning reuses pre-trained models....
  3. [0.034] Generative AI creates new content from learned patterns....

Query 2: How do computers understand language?
Embedding: [-0.315, 0.850, ...]
Top matches:
  1. [0.084] Generative AI creates new content from learned patterns....
  2. [0.062] Reinforcement learning trains agents through rewards....
  3. [0.060] Deep learning uses neural networks with multiple layers....

Query 3: Tell me about AI training methods.
Embedding: [3.189, -1.783, ...]
Top matches:
  1. [0.126] Transfer learning reuses pre-trained models....
  2. [0.088] Machine learning is a subset of artificial intel

In [ ]:
from hypernodes import DualNode, node

In [2]:
from typing import List


class Encoder:
    def __init__(self, model_name: str):
        self.model_name = model_name

    def encode(self, text: str) -> List[float]:
        return [0.1, 0.2, 0.3]

    def encode_batch(self, texts: List[str]) -> List[List[float]]:
        return [[0.1, 0.2, 0.3] for _ in texts]


In [3]:
@node(output_name="encoded_text")
def process(text: str, encoder: Encoder) -> List[float]:
    return encoder.encode(text)


pipeline = Pipeline(nodes=[process])

In [4]:
pipeline.visualize()

In [5]:
results = pipeline.run(
    inputs={"text": "Hello, world!", "encoder": Encoder(model_name="test")}
)

In [6]:
results

{'encoded_text': [0.1, 0.2, 0.3]}

In [7]:
results = pipeline.map(
    inputs={
        "text": ["Hello, world!", "Hey! World"],
        "encoder": Encoder(model_name="test"),
    },
    map_over="text",
)

# DualNode Example: Batch-Optimized Nodes

DualNode allows you to define two implementations:
- **singular**: For single-item execution (used in `.run()` and type hints)
- **batch**: For optimized batch execution (used in `.map()` with DaftEngine)


In [8]:
try:
    from daft import Series
except ImportError:
    Series = list  # Fallback for type hints


# Define singular and batch functions
def encode_singular(text: str, encoder: Encoder) -> List[float]:
    """Process single item"""
    return encoder.encode(text)


def encode_batch(texts: Series, encoder: Encoder) -> Series:
    """Process batch of items - optimized"""
    return Series.from_pylist([encoder.encode(t) for t in texts.to_pylist()])


# Create DualNode
dual_encode_node = DualNode(
    output_name="encoded_text",
    singular=encode_singular,
    batch=encode_batch,
)


In [9]:
# Create pipeline with DualNode
dual_pipeline = Pipeline(nodes=[dual_encode_node])
dual_pipeline.visualize()


In [10]:
# Test with .run() - uses singular function
encoder = Encoder(model_name="test")
result = dual_pipeline.run(inputs={"text": "Hello, world!", "encoder": encoder})
print("Single item result:", result)


Single item result: {'encoded_text': [0.1, 0.2, 0.3]}


In [11]:
# Test with .map() - uses batch function (optimized!)
results = dual_pipeline.map(
    inputs={
        "text": ["Hello", "World", "Test"],
        "encoder": encoder,
    },
    map_over="text",
)
print("Batch results:", results)


Batch results: [{'encoded_text': [0.1, 0.2, 0.3]}, {'encoded_text': [0.1, 0.2, 0.3]}, {'encoded_text': [0.1, 0.2, 0.3]}]


In [12]:
results

[{'encoded_text': [0.1, 0.2, 0.3]},
 {'encoded_text': [0.1, 0.2, 0.3]},
 {'encoded_text': [0.1, 0.2, 0.3]}]